Import Statements

In [1]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq
import numpy as np

Loading processed data from 03ipynb

In [2]:
month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.parquet"

table = pq.read_table(path)
ts_data = table.to_pandas()
ts_data[ts_data["pickup_location_id"]==43].head(18)

,pickup_hour,pickup_location_id,rides
29760,2023-01-01 00:00:00,43,92
29761,2023-01-01 01:00:00,43,81
29762,2023-01-01 02:00:00,43,29
29763,2023-01-01 03:00:00,43,15
29764,2023-01-01 04:00:00,43,4
29765,2023-01-01 05:00:00,43,4
29766,2023-01-01 06:00:00,43,3
29767,2023-01-01 07:00:00,43,12
29768,2023-01-01 08:00:00,43,12
29769,2023-01-01 09:00:00,43,23


Function that melts the dataframe in a specific way to create target variable (from lead values)

In [3]:
def transform_time_series_to_tabular(df, location_id, feature_col="rides", window_size=12, step_size=1):
    """
    Transforms time series data for a given location ID into a tabular format.
    The first `window_size` rows are used as features, and the next row is the target.
    The process slides down by `step_size` rows at a time to create the next set of features and target.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing time series data.
        location_id (int): The location ID to filter the data for.
        feature_col (str): The column name containing the values to use as features and target (default is "rides").
        window_size (int): The number of rows to use as features (default is 12).
        step_size (int): The number of rows to slide the window by (default is 1).

    Returns:
        pd.DataFrame: A transformed DataFrame where the first `window_size` columns are features
                      and the last column is the target.
    """
    # Filter the data for the given location ID
    location_data = df[df["pickup_location_id"] == location_id].reset_index(drop=True)

    # Extract the feature column as a NumPy array
    values = location_data[feature_col].values

    # Ensure there are enough rows to create at least one window
    if len(values) <= window_size:
        raise ValueError("Not enough data to create even one window of features and target.")

    # Create the tabular data using a sliding window approach
    rows = []
    for i in range(0, len(values) - window_size, step_size):
        # The first `window_size` values are features, and the next value is the target
        features = values[i:i + window_size] 
        target = values[i + window_size] 
        rows.append(np.append(features, target))

    # Convert the list of rows into a DataFrame
    column_names = [f"feature_{i+1}" for i in range(window_size)] + ["target"]
    transformed_df = pd.DataFrame(rows, columns=column_names)

    return transformed_df

features_targets = transform_time_series_to_tabular(ts_data, 43, "rides", 24, 1)
#Running the function on our loaded processed dataset to create target
features_targets.head(10)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,target
0,92,81,29,15,4,4,3,12,12,23,...,119,103,65,39,35,32,40,18,13,2
1,81,29,15,4,4,3,12,12,23,37,...,103,65,39,35,32,40,18,13,2,0
2,29,15,4,4,3,12,12,23,37,41,...,65,39,35,32,40,18,13,2,0,2
3,15,4,4,3,12,12,23,37,41,102,...,39,35,32,40,18,13,2,0,2,2
4,4,4,3,12,12,23,37,41,102,97,...,35,32,40,18,13,2,0,2,2,0
5,4,3,12,12,23,37,41,102,97,106,...,32,40,18,13,2,0,2,2,0,1
6,3,12,12,23,37,41,102,97,106,119,...,40,18,13,2,0,2,2,0,1,5
7,12,12,23,37,41,102,97,106,119,103,...,18,13,2,0,2,2,0,1,5,10
8,12,23,37,41,102,97,106,119,103,65,...,13,2,0,2,2,0,1,5,10,16
9,23,37,41,102,97,106,119,103,65,39,...,2,0,2,2,0,1,5,10,16,29
